##### CLASSIFICATION (Target = Categorie/Testo)
La scelta del parametro `scoring` determina come l'algoritmo valuta se un modello è "bravo" o "scarso". Ecco una legenda per orientarsi tra le opzioni disponibili in Scikit-Learn.



| Stringa (`scoring=...`) | Quando usarla (Best Use Case) | Descrizione Tecnica |
| :--- | :--- | :--- |
| **`'accuracy'`** | **Dati Bilanciati** | Percentuale di risposte esatte sul totale. <br> *Pericolosa su dataset sbilanciati (vedi Paradosso dell'Accuratezza).* |
| **`'recall_macro'`** | **Dati Sbilanciati (Focus su Errori)** | Calcola la Recall per ogni classe e ne fa la media semplice. <br> *Tratta la classe minuscola (Classe 2) con la stessa importanza della classe gigante (Classe 0).* |
| **`'recall_weighted'`** | Classi Sbilanciati (Focus su Generale) | Media delle Recall pesata in base al numero di campioni. <br> *Le classi giganti dominano il punteggio finale.* |
| **`'precision_macro'`** | Evitare Falsi Positivi | Come la recall_macro, ma guarda la precisione. Utile se il costo di un "Falso Allarme" è alto. |
| **`'f1_macro'`** | **Bilancio P/R** | Media armonica tra Precision e Recall. <br> *Spesso il miglior compromesso generale per problemi multi-classe sbilanciati.* |
| **`'roc_auc_ovr'`** | Probabilità & Ranking | Area sotto la curva (One-vs-Rest). Valuta quanto bene il modello separa le classi, indipendentemente dalla soglia di decisione. |

Dato che il tuo problema principale è che il modello "ignora" la Classe 2 (Recall bassa su quella classe), utilizzare **`scoring='recall_macro'`** oppure **`scoring='f1_macro'`** obbliga la Grid Search a cercare iperparametri che migliorino le prestazioni anche sulle classi più piccole, penalizzando i modelli che le ignorano.

##### Metriche di Regressione (Target = Numeri Continui)

Quando utilizziamo `GridSearchCV` o `cross_val_score` per problemi di Regressione (dove il target è un numero continuo), dobbiamo fare attenzione a una convenzione specifica di Scikit-Learn.

##### 1. Il Concetto: Perché "Negative"?
Scikit-Learn è progettato come un ottimizzatore che cerca sempre di **MASSIMIZZARE** il punteggio (Score).
* Per l'Accuratezza, più alto è meglio ($90\% > 80\%$). Tutto ok.
* Per l'Errore (MSE, RMSE), più **basso** è meglio ($5€ < 10€$ di errore).

Per non riscrivere tutto il codice interno, Scikit-Learn applica un trucco matematico:
> **Trasforma l'Errore in un numero Negativo.**
* Errore 5 $\rightarrow$ Score -5
* Errore 10 $\rightarrow$ Score -10
Poiché $-5$ è matematicamente più grande (più a destra sulla retta numerica) di $-10$, l'algoritmo cercando il "massimo" troverà correttamente l'errore più basso.



##### 2. Tabella di Conversione (Cheat Sheet)

Ecco le stringhe esatte da inserire nel parametro `scoring=...`:

| Metrica Richiesta | Stringa Scikit-Learn | Descrizione Tecnica |
| :--- | :--- | :--- |
| **RMSE** (Root Mean Squared Error) | **`'neg_root_mean_squared_error'`** | La radice dell'errore quadratico medio. <br> *È la più usata perché è nella stessa unità di misura del target (es. Euro).* |
| **MSE** (Mean Squared Error) | **`'neg_mean_squared_error'`** | L'errore al quadrato. <br> *Penalizza enormemente gli errori grandi (Outliers).* |
| **MAE** (Mean Absolute Error) | **`'neg_mean_absolute_error'`** | La media degli errori assoluti. <br> *Più robusto se ci sono outliers strani, perché non eleva al quadrato.* |
| **$R^2$ Score** (Coefficiente di Det.) | **`'r2'`** | **Eccezione:** Qui NON serve `neg_`. <br>Indica quanta varianza è spiegata dal modello (1.0 = Perfetto, 0.0 = Pessimo). Più alto è, meglio è. |

##### Nota Operativa
Quando stampi i risultati `grid.best_score_`, vedrai un numero negativo (es. `-3.45`).
**Ignora il meno!** Il vero errore RMSE è semplicemente `3.45`.